In [1]:
import sys
from pathlib import Path

# Add the project src folder to Python's path
root = Path().resolve().parent
print(root)
sys.path.append(str(root))

C:\Users\Ariel\Projects\email-security-copilot


In [2]:
import os, pandas as pd
from src.email_security_copilot.config import ProjectConfig
from src.email_security_copilot.data.data import EmailDataset
from src.email_security_copilot.data.textPreprocessor import TextPreprocessor
# from src.utils import combine_subject_content
from src.email_security_copilot.data.dataset import SpamFrameDataset

c:\Users\Ariel\Projects\email-security-copilot\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
cfg = ProjectConfig()
cfg.validate()
print("Config OK")

Config OK


In [5]:
spam_dir = cfg.data.spam_dir
ham_dir  = cfg.data.ham_dir

assert os.path.isdir(spam_dir), f"Missing dir: {spam_dir}"
assert os.path.isdir(ham_dir),  f"Missing dir: {ham_dir}"

spam_df = EmailDataset(spam_dir, label="spam").load()
ham_df  = EmailDataset(ham_dir,  label="ham").load()
df = pd.concat([spam_df, ham_df], ignore_index=True)

print(
    f"Loaded: {len(df)} emails "
    f"({(df.label=='spam').sum()} spam, {(df.label=='ham').sum()} ham)"
)

Loaded: 3948 emails (1397 spam, 2551 ham)


In [27]:

from email_security_copilot.data.data import load_data

df_test = load_data(spam_dir=str(spam_dir), ham_dir=str(ham_dir))

In [28]:
df_test

,file,sender,recipient,cc,num_cc,subject,content,num_hops,label
0,00001.317e78fa8ee2f54cd4890fdc09ba8176,Start Now <startnow2002@hotmail.com>,ilug@linux.ie,,0,[ILUG] STOP THE MLM INSANITY,Greetings!\n\nYou are receiving this letter be...,6,spam
1,00002.9438920e9a55591b18e60d1ed37d992b,lmrn@mailexcite.com,ranmoore@cybertime.net,,0,"Real Protection, Stun Guns! Free Shipping! Ti...","The Need For Safety Is Real In 2002, You Might...",3,spam
2,00003.590eff932f8704d8b0fcbe69d023b54d,amknight@mailexcite.com,cbmark@cbmark.com,,0,"New Improved Fat Burners, Now With TV Fat Abso...","*****Bonus Fat Absorbers As Seen On TV, Includ...",3,spam
3,00004.bdcc075fa4beb5157b5dd6cd41d8887b,jordan23@mailexcite.com,ranmoore@swbell.net,,0,"New Improved Fat Burners, Now With TV Fat Abso...","*****Bonus Fat Absorbers As Seen On TV, Includ...",4,spam
4,00005.ed0aba4d386c5e62bc737cf3f0ed9589,yyyy@pluriproj.pt,yyyy@pluriproj.pt,,0,"Never Repay Cash Grants, $500 - $50,000, Secre...","Government Grants E-Book 2002\nedition, Just $...",4,spam
...,...,...,...,...,...,...,...,...,...
3943,2547.18ce2b2d4f757e70154b5c8cfe6e684f,newscientist <rssfeeds@example.com>,yyyy@example.com,,0,Gene technique reveals human evolution,"URL: http://www.newsisfree.com/click/-3,871436...",3,ham
3944,2548.b2259f104edd1ecbc9dc0327df387147,guardian <rssfeeds@example.com>,yyyy@example.com,,0,Go-ahead for new-style hospitals,"URL: http://www.newsisfree.com/click/-4,872399...",3,ham
3945,2549.d05fe143eb28d1fb19fbdc39781d5e34,newscientist <rssfeeds@example.com>,yyyy@example.com,,0,Malicious code hidden in email software,"URL: http://www.newsisfree.com/click/-3,871080...",3,ham
3946,2550.963956a866a2e00b3d5b353ecc9216ae,guardian <rssfeeds@example.com>,yyyy@example.com,,0,'Flexible' retirement gains ground,"URL: http://www.newsisfree.com/click/-4,872400...",3,ham


In [19]:
from src.email_security_copilot.features.features import EmailFeatureClass
features_df = EmailFeatureClass()(df)

In [26]:
cols = ['sender', 'recipient', 'num_cc', 'subject', 'content', 
        'subject_len', 'content_len', 'has_link', 'num_hops', 'label']
raw_df = features_df[cols]

In [ ]:
# df["text_raw"] = df.apply(combine_subject_content, axis=1)

# pre = TextPreprocessor(cfg)
# df["text_raw_clean"] = df["text_raw"].fillna("").map(pre)

# assert df["text_raw_clean"].isna().sum() == 0, "NaNs in cleaned content"
# non_empty_rate = (df["text_raw_clean"].str.len() > 0).mean()
# print(f"Cleaned text — non-empty rate: {non_empty_rate:.2%}")

# df_clean = df[["text_raw_clean", "label"]].copy()

Loaded: 3948 emails (1397 spam, 2551 ham)
Cleaned text — non-empty rate: 100.00%


In [ ]:
# llm_dataset = SpamFrameDataset(
#     df=df_clean,
#     cfg=cfg,
# )
# len(llm_dataset), llm_dataset[0].keys()

(3948, dict_keys(['input_ids', 'attention_mask', 'labels']))

In [ ]:
# from src.utils import combine_subject_content
# df["text_raw"] = df.apply(combine_subject_content, axis=1)

# pre = TextPreprocessor(cfg)
# df["text_raw_clean"] = df["text_raw"].fillna("").map(pre)

# assert df["text_raw_clean"].isna().sum() == 0, "NaNs in cleaned content"
# non_empty_rate = (df["text_raw_clean"].str.len() > 0).mean()
# print(f"Cleaned text — non-empty rate: {non_empty_rate:.2%}")

Cleaned text — non-empty rate: 100.00%
